In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('Split_smol/train', target_size=(224, 224), batch_size=32, class_mode='categorical')

validation_generator = test_datagen.flow_from_directory('Split_smol/val', target_size=(224, 224), batch_size=32, class_mode='categorical')

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'Split_smol/train'

In [ ]:
from tensorflow.keras.applications import VGG16

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(255, 255, 3))

for layer in base_model.layers:
  layer.trainable = False

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense


model = Sequential()
model.add((base_model))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation="relu"))
model.add(keras.layers.Dense(11, activation="softmax"))

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(train_generator, batch_size=16, epochs=5, validation_data=validation_generator, validation_batch_size=16)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(5, 5), dpi=100)

plt.plot(history.history['accuracy'], label="Training Accuracy")
plt.plot(history.history['val_accuracy'], label="Validation Accuracy")
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.grid()
plt.legend()
plt.show()

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

def preprocess_image(image_path, target_size=(225, 225)):
    img = load_img(image_path, target_size=target_size)
    x = img_to_array(img)
    x = x.astype('float32') / 255.
    x = np.expand_dims(x, axis=0)
    return x

def preprocess_image2(image_path, target_size=(225, 225)):
    img = load_img(image_path, target_size=target_size)
    x = img_to_array(img)
    x = x.astype('float32') / 255.
    return x

x = preprocess_image('Dataset/Test/Test/Rust/82f49a4a7b9585f1.jpg')

predictions = model.predict(x)
predictions[0]

In [ ]:
labels = train_generator.class_indices
labels = {value: key for key, value in labels.items()}
labels

In [ ]:
predicted_label = labels[np.argmax(predictions)]
print(predicted_label)

In [ ]:
test_generator = test_datagen.flow_from_directory('Dataset/Validation/Validation', target_size=(225, 225), batch_size=32, class_mode='categorical')

In [ ]:
y_test = test_generator.class_indices
y_test = {value: key for key, value in y_test.items()}
y_test = y_test[np.argmax]

In [ ]:
import os

def load_data(data_dir):
  
  X = []  
  y = [] 

  for subdir in os.listdir(data_dir):

    subdirectory_path = os.path.join(data_dir, subdir)

    if os.path.isdir(subdirectory_path):

      label = 0
      
      if (subdir == "Healthy"):
        label = 0
      elif (subdir == "Powdery"):
        label = 1
      elif (subdir == "Rust"):
        label = 2

      for filename in os.listdir(subdirectory_path):

        image_path = os.path.join(subdirectory_path, filename)

        image = preprocess_image2(image_path) 

        X.append(image)
        y.append(label)

  X = np.array(X)
  y = np.array(y)

  return X, y

In [ ]:
x_test, y_test = load_data("Dataset/Test/Test")

In [ ]:
prediction_test = model.predict(x_test)
prediction_test = np.argmax(prediction_test, axis=1)

In [ ]:
from sklearn.metrics import accuracy_score

print("Test set accuracy: ", accuracy_score(y_test, prediction_test))

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

with open('skinModel.tflite', 'wb') as f:
  f.write(tflite_model)